### AutoGen Team State Management 

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.base import TaskResult
from autogen_core import CancellationToken

import os

from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

if not all([azure_openai_api_key, azure_openai_endpoint, azure_openai_deployment_name, azure_openai_api_version]):
    raise ValueError("""
        Please set all required environment variables: 
            AZURE_OPENAI_API_KEY, 
            AZURE_OPENAI_ENDPOINT, 
            AZURE_OPENAI_DEPLOYMENT_NAME, 
            AZURE_OPENAI_API_VERSION
        """)


In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a fun fact expert about animals. ",
)

team = RoundRobinGroupChat(
    [assistant_agent],
    termination_condition=MaxMessageTermination(2),
)

stream = team.run_stream(task = "Tell me a surprising fun fact about Octopuses.")

await Console(stream)

In [ ]:
team_state = await team.save_state()
print("Team state saved successfully.")

In [ ]:
print("Team state:", team_state)

In [ ]:
await team.reset()

stream = team.run_stream(task = "What was the last animal fact you shared?")

await Console(stream)

In [ ]:
await team.load_state(team_state)

print("Team state loaded successfully.")

stream = team.run_stream(task = "What was the last animal fact you shared?")

await Console(stream)

In [ ]:
import json

with open("team_state.json", "w") as f:
    json.dump(team_state, f, indent=4, sort_keys=True, ensure_ascii=False, default=str)
    
print("Team state saved to team_state.json successfully.")

In [ ]:
with open("team_state.json", "r") as f:
    loaded_state = json.load(f)
    
new_agent_team = RoundRobinGroupChat(
    [assistant_agent],
    termination_condition=MaxMessageTermination(2),
)

await new_agent_team.load_state(loaded_state)

stream = team.run_stream(task = "What was the last animal fact you shared?")

await Console(stream)